In [1]:
%pip install langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()  # loads GOOGLE_API_KEY from .env

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

response = llm.invoke("I want to open a cafe. Suggest some fancy names for it.")
print(response.content)


## Fancy Cafe Names:

**Evocative & Atmospheric:**

* The Gilded Bean
* Ember & Oak
* The Velvet Spoon
* Whispering Pines Cafe
* Moonlit Brew
* Crimson Cup
* The Copper Kettle
* The Alchemist's Brewhouse
* The Secret Garden Cafe
* Starlit Sips

**Elegant & Sophisticated:**

* La Belle Époque Cafe
* The Grand Cafe
* Atelier de Café
* The Parisian Patisserie
* The Ivory Tower
* The Sterling Spoon
* Maison du Café
* The Royal Bean
* Chateau de Café
*  The Gilded Lily

**Unique & Playful:**

* The Caffeine Patch
* The Daily Grind (with a sophisticated logo/design)
* The Buzz Stop
* The Coffee Alchemist
* The Bean Scene
*  Brewtiful Beginnings
* The Cozy Kettle
*  Percolations
* The Daily Dose
*  Ampersand & Espresso


**Location Specific (adapt to your area):**

* [Your Street Name] Coffee House
* [Landmark] Cafe
* [Neighborhood] Grind


**Tips for choosing:**

* **Check for availability:** Make sure the name isn't already in use (website domain, social media handles, etc.).
* **Consider y

In [4]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=["cuisine", "vibe"],
    template="I want to open a {vibe} cafe that serves {cuisine} food. Suggest a fancy name for it."
)

print(prompt_template_name.format(cuisine="Japanese", vibe="cozy"))



I want to open a cozy cafe that serves Japanese food. Suggest a fancy name for it.


In [5]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run(cuisine="Italian", vibe="modern")

"**Emphasizing Italian Heritage:**\n\n*  Casa Amante (Lover's House)\n*  La Dolce Vita Caffè\n*  Il Giardino Segreto (The Secret Garden)\n*  Piazza Emilia\n*  Trattoria Moderna\n*  Bottega del Caffè (Coffee Shop)\n*  Aroma di Roma (Scent of Rome)\n\n\n**Emphasizing Modernity & Chic:**\n\n*  Aperitivo & Co.\n*  The Espresso Bar\n*  Caffè Elegante\n*  Lineage Coffee & Cucina\n*  The Crafted Plate\n*  Artisan Espresso\n*  Miele & Latte (Honey & Milk)\n\n\n**Creative & Evocative Names:**\n\n*  The Golden Ratio\n*  Ember & Oak\n*  The Amalfi Coast Cafe\n*  Cinque Terre Kitchen\n*  Bellissimo Bites\n*  A Tavola (To the Table)\n\n\n**Tips for Choosing:**\n\n* **Check for availability:** Make sure the name isn't already in use (website domain, social media handles).\n* **Keep it concise and memorable:**  Shorter names are easier to remember and brand.\n* **Consider your target audience:**  A sophisticated name might attract a different clientele than a more playful one.\n* **Say it out loud:**

In [6]:
# trying simple sequential search

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)
prompt_template_name = PromptTemplate(
    input_variables=["cuisine", "vibe"],
    template="I want to open a {vibe} cafe that serves {cuisine} food. Suggest only one fancy name for it."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['cafe_name'],
    template = """Suggest some menu items for {cafe_name}. Return it as a comma separated list"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)


In [9]:
# Sequentially run the chains using a function since SimpleSequentialChain requires single input

def get_cafe_name_and_menu(cuisine, vibe):
	cafe_name = name_chain.run(cuisine=cuisine, vibe=vibe)
	menu_items = food_items_chain.run(cafe_name=cafe_name)
	return cafe_name, menu_items

cafe_name, menu_items = get_cafe_name_and_menu("Indian", "luxurious")
print("Cafe Name:", cafe_name)
print("Menu Items:", menu_items)

Cafe Name: Amber & Saffron
Menu Items: Spiced Lamb Tagine, Saffron Risotto, Amber Ale-Braised Short Ribs, Saffron-Infused Crème Brûlée,  Amber-Glazed Salmon, Saffron & Pistachio Cake,  Spiced Carrot & Chickpea Salad,  Amber Wheat Beer Bread,  Saffron-Scented Chicken,  Date & Saffron Pudding


In [15]:
# trying sequential search

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)
prompt_template_name = PromptTemplate(
    input_variables=["cuisine", "vibe"],
    template="I want to open a {vibe} cafe that serves {cuisine} food. Suggest only one fancy name for it."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="cafe_name")

prompt_template_items = PromptTemplate(
    input_variables = ['cafe_name'],
    template = """Suggest some menu items for {cafe_name}. Return it as a comma separated list"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")


In [17]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["cuisine", "vibe"],
    output_variables=["cafe_name", "menu_items"],
    verbose=True
)
result = chain.invoke({"cuisine": "Mexican", "vibe": "vibrant"})
print(result)



> Entering new SequentialChain chain...

> Finished chain.
{'cuisine': 'Mexican', 'vibe': 'vibrant', 'cafe_name': 'Sol y Sombra (Sun and Shade)', 'menu_items': 'Gazpacho, Paella, Tapas Trio, Churros con Chocolate, Sangria,  Ensalada Española,  Gambas al Ajillo,  Tortilla Española,  Crema Catalana,  Iberico Ham Croquettes'}
